In [46]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('com1').getOrCreate()
df = spark.read.csv("BankChurners.csv",header=True,inferSchema=True)
df.count()

10127

In [47]:
df1 = df.repartition(4)

In [48]:
df1.rdd.getNumPartitions()

4

In [33]:
from pyspark.sql.functions import spark_partition_id

df2 = df1.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count()
df2.show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0| 2531|
|          1| 2532|
|          2| 2532|
|          3| 2532|
+-----------+-----+



### repartitioned By using Column name

In [58]:
df4 = df1.repartition(8, 'Card_category')*+

In [59]:
df4.rdd.getNumPartitions()

8

In [60]:
from pyspark.sql.functions import spark_partition_id

df5 = df4.withColumn("partitionId",spark_partition_id()).groupBy("partitionId").count()
df5.show()

+-----------+-----+
|partitionId|count|
+-----------+-----+
|          0| 9436|
|          2|  116|
|          6|  575|
+-----------+-----+



In [42]:
####### 2.Another Example
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('com1').getOrCreate()
d1 = spark.read.csv("data_skew_file.csv",header=True)
d= d1.repartition(6)
d.show()
d.rdd.getNumPartitions()

from pyspark.sql.functions import spark_partition_id, asc, desc

d = d.withColumn("partition_id",spark_partition_id())
df2 = d.withColumn("partition_id", spark_partition_id())\
      .groupBy("partition_id")\
      .count()\
      .orderBy(desc("count"))
df2.show()

+----+------+
|name| value|
+----+------+
|   x|    66|
|   x|    33|
|   x|    89|
|   x|    77|
|   x|    88|
|   z|    93|
|   x|    44|
|   x|    22|
|   c|     3|
|   y|    92|
|   x|    88|
|   x|    90|
|   x|    44|
|   x|    22|
|   b|     2|
|   x|    11|
|   y|    91|
|   x|    90|
|   d|     4|
|   x|    44|
+----+------+
only showing top 20 rows

+------------+-----+
|partition_id|count|
+------------+-----+
|           3|    7|
|           0|    6|
|           2|    6|
|           1|    6|
|           4|    6|
|           5|    6|
+------------+-----+



In [44]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, concat, explode, floor, lit, rand, array

spark = SparkSession.builder.appName("RemoveDataSkewness").master("local[*]").getOrCreate()

data1 = [
    ("x", "bc"),
    ("x", "ce"),
    ("x", "ab"),
    ("x", "ef"),
    ("x", "gh"),
    ("y", "hk"),
    ("z", "jk")
]
columns1 = ["_1", "_2"]
df1 = spark.createDataFrame(data1, columns1)
df1.show(10, False)

data2 = [
    ("x", "gkl"),
    ("y", "nmb"),
    ("z", "qwe")
]
columns2 = ["_1", "_2"]
df2 = spark.createDataFrame(data2, columns2)
df2.show(10, False)

# Method to eliminate data skewness
def eliminate_data_skew(left_table, left_col, right_table):
    df1 = left_table.withColumn(
        left_col,
        concat(left_table[left_col], lit("_"), floor(rand() * 10))
    )
    df2 = right_table.withColumn(
        "explodedCol",
        explode(array([lit(i) for i in range(11)]))
    )
    return df1, df2

df3, df4 = eliminate_data_skew(df1, "_1", df2)

df3.show(100, False)
df4.show(100, False)

# Join after eliminating data skewness
result = df3.join(
    df4,
    df3["_1"] == concat(df4["_1"], lit("_"), df4["explodedCol"])
)

result.show(100, False)


+---+---+
|_1 |_2 |
+---+---+
|x  |bc |
|x  |ce |
|x  |ab |
|x  |ef |
|x  |gh |
|y  |hk |
|z  |jk |
+---+---+

+---+---+
|_1 |_2 |
+---+---+
|x  |gkl|
|y  |nmb|
|z  |qwe|
+---+---+

+---+---+
|_1 |_2 |
+---+---+
|x_5|bc |
|x_9|ce |
|x_5|ab |
|x_6|ef |
|x_4|gh |
|y_9|hk |
|z_6|jk |
+---+---+

+---+---+-----------+
|_1 |_2 |explodedCol|
+---+---+-----------+
|x  |gkl|0          |
|x  |gkl|1          |
|x  |gkl|2          |
|x  |gkl|3          |
|x  |gkl|4          |
|x  |gkl|5          |
|x  |gkl|6          |
|x  |gkl|7          |
|x  |gkl|8          |
|x  |gkl|9          |
|x  |gkl|10         |
|y  |nmb|0          |
|y  |nmb|1          |
|y  |nmb|2          |
|y  |nmb|3          |
|y  |nmb|4          |
|y  |nmb|5          |
|y  |nmb|6          |
|y  |nmb|7          |
|y  |nmb|8          |
|y  |nmb|9          |
|y  |nmb|10         |
|z  |qwe|0          |
|z  |qwe|1          |
|z  |qwe|2          |
|z  |qwe|3          |
|z  |qwe|4          |
|z  |qwe|5          |
|z  |qwe|6          |
|z 